In [1]:
import pandas as pd
from pydantic import BaseModel
from PandasToPydantic.src.dataframeSerializer import expandAnnotation, getBaseFields, getListFields, serializeDataframe


In [2]:
class ParentModel(BaseModel):
    parentString: str
    parentInteger: int
    parentFloat: float
    parentListString: list[str]



In [3]:
hasattr(ParentModel, "__annotations__")

True

In [4]:
expandAnnotation(ParentModel)

{'parentString': str,
 'parentInteger': int,
 'parentFloat': float,
 'parentListString': [str]}

In [2]:
class DataPoint(BaseModel):
    DataName: str
    DataType: str
    DataValue: str

class DataCollection(BaseModel):
    CollectionName: str
    CollectionDescription: str
    DataPointList: list[DataPoint]

class CollectionOperation(BaseModel):
    OperationName: str
    OperationUser: str
    OperationDate: str
    DataCollectionList: list[DataCollection]

In [16]:
expandAnnotation(DataPoint)

{'DataName': str, 'DataType': str, 'DataValue': str}

In [13]:
data = pd.read_csv("PandasToPydantic/test/testData.csv")

In [15]:
serializeDataframe(data, expandAnnotation(CollectionOperation))


[{'OperationName': 'Operation 1',
  'OperationUser': 'User 1',
  'OperationDate': '2023-01-01',
  'DataCollectionList': [{'CollectionName': 'Collection 1',
    'CollectionDescription': 'Description 1',
    'DataPointList': [{'DataName': 'Name 1',
      'DataType': 'Type 1',
      'DataValue': 'Value 2'},
     {'DataName': 'Name 2', 'DataType': 'Type 2', 'DataValue': 'Value 1'},
     {'DataName': 'Name 3', 'DataType': 'Type 2', 'DataValue': 'Value 1'}]},
   {'CollectionName': 'Collection 2',
    'CollectionDescription': 'Description 2',
    'DataPointList': [{'DataName': 'Name 1',
      'DataType': 'Type 1',
      'DataValue': 'Value 2'},
     {'DataName': 'Name 2', 'DataType': 'Type 2', 'DataValue': 'Value 1'}]}]},
 {'OperationName': 'Operation 2',
  'OperationUser': 'User 2',
  'OperationDate': '2023-02-02',
  'DataCollectionList': [{'CollectionName': 'Collection 1',
    'CollectionDescription': 'Description 1',
    'DataPointList': [{'DataName': 'Name 1',
      'DataType': 'Type 1',


In [6]:
expandAnnotation(CollectionOperation)["DataCollectionList"]

[{'CollectionName': str,
  'CollectionDescription': str,
  'DataPointList': [{'DataName': str, 'DataType': str, 'DataValue': str}]}]